# Install library

In [31]:
!pip install requests
!pip install psycopg2

In [50]:
#Download Librarys
import psycopg2
import requests
import pandas as pd
from psycopg2.extras import execute_values

# Function Extract Data
def extract_data():

  url = 'https://tradestie.com/api/v1/apps/reddit'
  headers = {"Accept-Encoding": "gzip, deflate"}

  response = requests.get(url, headers=headers)
  data = response.json()
  df = pd.DataFrame(data)

  return df


# Conexion a la base de datos
def connection_db():

  url="data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com"
  data_base="data-engineer-database"
  user="fblasfacundo_coderhouse"


  try:
      conn = psycopg2.connect(
          host='data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com',
          dbname=data_base,
          user=user,
          password='cU7tBjTI10',
          port='5439'
      )
      print("Connected to Redshift successfully!")

      return conn

  except Exception as e:
      print("Unable to connect to Redshift.")
      print(e)

#Funcion para ETL
def cargar_en_redshift(conn, table_name, dataframe):
    dtypes= dataframe.dtypes
    cols= list(dtypes.index )
    print(cols)
    tipos= list(dtypes.values)
    type_map = {'int64': 'INT','int32': 'INT','float64': 'FLOAT','object': 'VARCHAR(50)','bool':'BOOLEAN'}
    sql_dtypes = [type_map[str(dtype)] for dtype in tipos]
    # Definir formato SQL VARIABLE TIPO_DATO
    column_defs = [f"{name} {data_type}" for name, data_type in zip(cols, sql_dtypes)]
    # Combine column definitions into the CREATE TABLE statement
    table_schema = f"""
        CREATE TABLE IF NOT EXISTS {table_name} (
            {', '.join(column_defs)}
        )
        SORTKEY (sentiment);
        """
    #Crear la tabla
    cur = conn.cursor()
    cur.execute(table_schema)
    # Generar los valores a insertar
    values = [tuple(x) for x in dataframe.to_numpy()]
    # Definir el INSERT
    insert_sql = f"INSERT INTO {table_name} ({', '.join(cols)}) VALUES %s"
    # Execute the transaction to insert the data
    cur.execute("BEGIN")
    execute_values(cur, insert_sql, values)
    cur.execute("COMMIT")
    print('Proceso terminado')




# Enviando tabla a Redshift

In [51]:
if __name__ == '__main__':

  datab = extract_data()

  conexion_db = connection_db()

  cargar_en_redshift(conn=conexion_db, table_name='trade_data', dataframe= datab)

Connected to Redshift successfully!
['no_of_comments', 'sentiment', 'sentiment_score', 'ticker']
Proceso terminado
